In [15]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool,CatBoost, CatBoostRegressor
from collections import Counter
from keras.layers import Dense, Dropout, Embedding, Input, Reshape, Concatenate, Input, Flatten
from keras.models import Model

In [8]:
data_path = 'dataset/'
customer_train = pd.read_csv(f'{data_path}customer_train.csv')
customer_test = pd.read_csv(f'{data_path}customer_test.csv')
stories_description = pd.read_csv(f'{data_path}stories_description.csv')
stories_reaction_test = pd.read_csv(f'{data_path}stories_reaction_test.csv')
stories_reaction_test['event_dttm']=pd.to_datetime(stories_reaction_test['event_dttm'],infer_datetime_format=True)
stories_reaction_train = pd.read_csv(f'{data_path}stories_reaction_train.csv')
stories_reaction_train['event_dttm']=pd.to_datetime(stories_reaction_train['event_dttm'],infer_datetime_format=True)
transactions = pd.read_csv(f'{data_path}transactions.csv')


sample_submit = pd.read_csv(f'{data_path}sample_submit.csv')

In [9]:
train_and_val = stories_reaction_train.merge(customer_train, on='customer_id', how='left')
test = stories_reaction_test.merge(customer_test, on='customer_id', how='left')


In [10]:
train_dates = stories_reaction_train['event_dttm'].astype(np.int64)
test_dates = stories_reaction_test['event_dttm'].astype(np.int64)
period = train_dates.max() - (test_dates.max() - test_dates.min())

In [11]:
val = train_and_val[train_and_val.event_dttm.astype(np.int64) > period]
train = train_and_val[train_and_val.event_dttm.astype(np.int64) <= period]

In [12]:
len(train), len(val) 

(320248, 152893)

array([     15,      15,      15, ..., 1103495, 1103495, 1103495])

In [118]:
import keras.backend as K
def mertic_loss(y_true, y_pred):
    
    return -np.sum(y_true*y_pred*[-10,0.5,0.1,-0.1])

In [93]:
customers_count = len(set(train_and_val.customer_id.values))

In [94]:
def reaction_to_class(reaction):
    if reaction=="dislike":
        return np.array([1,0,0,0])
    if reaction=="like":
        return np.array([0,1,0,0])
    if reaction=="view":
        return np.array([0,0,1,0])
    if reaction=="skip":
        return np.array([0,0,0,1])
trainY = np.array(list(train.event.map(lambda x: reaction_to_class(x)).values))
valY = np.array(list(val.event.map(lambda x: reaction_to_class(x)).values))

In [83]:
len(val.customer_id.values)

152893

In [119]:
model_input = Input((1,), name='customer_id')
x = Embedding(customers_count, 32, input_length=1)(model_input)
x = Flatten()(x)
x = Dense(4, activation='softmax')(x)
model = Model(model_input, x)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc', mertic_loss])
print(model.summary())

Tensor("metrics_14/mertic_loss/strided_slice:0", shape=(?,), dtype=float32) Tensor("metrics_14/mertic_loss/strided_slice_1:0", shape=(4,), dtype=float32)
Model: "model_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
customer_id (InputLayer)     (None, 1)                 0         
_________________________________________________________________
embedding_21 (Embedding)     (None, 1, 32)             1312032   
_________________________________________________________________
flatten_20 (Flatten)         (None, 32)                0         
_________________________________________________________________
dense_20 (Dense)             (None, 4)                 132       
Total params: 1,312,164
Trainable params: 1,312,164
Non-trainable params: 0
_________________________________________________________________
None


In [120]:
batch_size=128
model.fit(x = train.customer_id.values, 
          y = trainY,
#           steps_per_epoch=1000,
          batch_size=batch_size,
          epochs=10,
          validation_data=(val.customer_id.values, valY),
#           validation_steps=len(val.customer_id.values)/batch_size
         )

Train on 320248 samples, validate on 152893 samples
Epoch 1/10
320248/320248 [==============================] - 6s 17us/step - loss: 1.1151 - acc: 0.5025 - mertic_loss: -0.0012 - val_loss: 0.9963 - val_acc: 0.5387 - val_mertic_loss: -0.0038
Epoch 2/10
320248/320248 [==============================] - 5s 17us/step - loss: 1.0356 - acc: 0.5053 - mertic_loss: -0.0043 - val_loss: 0.9852 - val_acc: 0.5385 - val_mertic_loss: -0.0046
Epoch 3/10
146688/320248 [============>.................] - ETA: 2s - loss: 1.0298 - acc: 0.5069 - mertic_loss: -0.0046

KeyboardInterrupt: 

In [115]:
predict = model.predict(train.customer_id.values)

In [117]:
-np.sum(trainY*predict*[-10,0.5,0.1,-0.1])

-5898.495399405433